# **MÓDULO 20 - Projeto de Credit Score - Naive Bayes**


No módulo 17 vocês realizaram a primeira etapa do projeto de crédito de vocês.
Então fizeram o tratamendo dos dados, balancearam as classes, transformaram as variáveis categóricas e separam base de treino e teste.
Nessa aula aplicaremos o algoritmo de naive bayes a base de vocês afim de tentarmos trazer previsões do score de crédito.

**IMPORTANTE:** Não se esqueçam de ao enviar o código de vocês para os tutores, enviarem as bases, pois como cada um de vocês realizou as alterações de tratamento indidualmente o tutor precisa ter acesso aos seus dados individuais.

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score


Durante a aula nossa variável a ser prevista (churn) continha apenas 2 categorias, a base de vocês contém mais. O Naive Bayes pode ser aplicado para problemas de classificação com múltiplas classes da mesma forma que para problemas de classificação binária. O Naive Bayes é um algoritmo de classificação probabilístico que calcula a probabilidade de uma amostra pertencer a cada classe e seleciona a classe com a maior probabilidade como a previsão final.
Em resumo, o Naive Bayes pode ser aplicado da mesma maneira para problemas de classificação com múltiplas classes, e os mesmos princípios se aplicam em termos de treinamento, avaliação e aplicação do modelo.

# 1) Comece carregando as bases de treino (X e y) e teste (X e y).
Verifique se o número de linhas condiz, se as variáveis estão corretas sendo apenas a de score para y e as demais nas bases de X e por último, se Y está balanceada no teste.

In [13]:
X_test = pd.read_csv("X_test.csv", delimiter=',')
X_train = pd.read_csv("X_train_balanced.csv", delimiter=',')
y_test = pd.read_csv("y_test.csv", delimiter=',')
y_train = pd.read_csv("y_train_balanced.csv", delimiter=',')

In [9]:
X_test

,Credit_Score,Idade,Tempo_Credito,Balanco,Qtd_Produtos,Possui_Cartao,Membro_Ativo,Salario_Anual,Genero_encoded,Pais_encoded
0,686,36.0,3,95848.58,2,0.0,1.0,7144694.0,1,0
1,703,35.0,1,0.00,1,1.0,1.0,14136462.0,1,0
2,730,26.0,6,0.00,1,1.0,1.0,1011687.0,1,2
3,675,33.0,6,0.00,2,0.0,1.0,1011687.0,0,0
4,754,24.0,3,121513.31,2,0.0,1.0,1011687.0,0,1
...,...,...,...,...,...,...,...,...,...,...
24922,786,37.0,4,0.00,2,0.0,1.0,1011687.0,0,2
24923,592,35.0,7,122724.50,1,1.0,1.0,17773874.0,1,0
24924,645,45.0,7,121286.32,1,1.0,1.0,869372.0,1,0
24925,567,47.0,1,126447.40,1,1.0,0.0,1011687.0,0,1


In [11]:
y_test

,Churn
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
24922,0.0
24923,0.0
24924,0.0
24925,1.0


In [12]:
X_train

,Credit_Score,Idade,Tempo_Credito,Balanco,Qtd_Produtos,Possui_Cartao,Membro_Ativo,Salario_Anual,Genero_encoded,Pais_encoded
0,711,35.000000,10,0.000000,2,1.000000,0.000000,5331178.0,1,0
1,619,33.000000,7,0.000000,2,0.000000,1.000000,1011687.0,1,0
2,841,54.000000,4,102721.490000,1,1.000000,1.000000,1011687.0,1,1
3,679,48.000000,6,125161.240000,1,1.000000,0.000000,579659.0,1,0
4,656,42.000000,6,0.000000,3,0.000000,0.000000,14233155.0,0,0
...,...,...,...,...,...,...,...,...,...,...
117945,529,43.000000,7,0.000000,1,0.514157,0.000000,1011687.0,1,2
117946,639,44.450575,7,0.000000,1,1.000000,0.725287,1011687.0,0,2
117947,541,49.890057,6,102744.063633,1,1.000000,0.000000,1011687.0,0,1
117948,593,37.911470,5,139949.785268,1,1.000000,0.000000,1011687.0,0,1


In [14]:
y_train

,Churn
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
117945,1.0
117946,1.0
117947,1.0
117948,1.0


Feito a verificação das bases de treino e teste e estão ok os numeros de linhas.

In [20]:
# Contagem de registros das clases de y_train
train_balance = y_train.value_counts()
print("Balanceamento de y_train:")
print(train_balance)
print("\n")

# Contagem de registros das clases de y_teste
# Balanceamento no teste não se faz, teste real
train_balance =y_test.value_counts()
print("Quntidade de registro de y_train:")
print(train_balance)

Balanceamento de y_train:
Churn
0.0      58975
1.0      58975
Name: count, dtype: int64


Quntidade de registro de y_train:
Churn
0.0      19649
1.0       5278
Name: count, dtype: int64


# 2) Aplique o algoritmo de Naive Bayes aos dados de treinamento.

In [21]:
# Instacia de classificação Naive Bayes Gaussiano
naive_bayes = GaussianNB()

# Treinamento do modelo
naive_bayes.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

# 3) Faça a avaliação do modelo com os dados de treinamento.
Traga a acurácia, recall e plote a matriz de confusão. Não se esqueça de avaliar com suas palavras o desempenho do modelo, interpretando as métricas.

Dica: Para calcularmos o recall em classificação multi classe precisamos usar o atributo macro:
recall = recall_score(y_train, y_pred_train, average='macro')

In [22]:
# Previssões para dados de treino
y_pred_train = naive_bayes.predict(X_train)
# Calcular a acurácia
accuracy = accuracy_score(y_train, y_pred_train)
print("Acurácia:", accuracy)


Acurácia: 0.5854938533276812


Verificado que o teste de Acurácia apresentou uma taxa de previsões corretas com 58%

In [32]:
conf_matrix = confusion_matrix(y_train, y_pred_train)

# Nomes das clases
class_name = ['Chun', 'Não Churn']

# Plot da matriz de confusão com Plotly
fig = ff.create_annotated_heatmap(
    z = conf_matrix,
    x = class_name,
    y = class_name,
    colorscale = 'Viridis',
    showscale = True,
)
fig.update_layout(
    title = 'Matriz de Confusão - Treino',
    xaxis_title = 'Predict',
    yaxis_title = 'True',
    font = dict(
        family = 'Arial',
        size = 12,
        color = 'black'
    )
)



Com a matriz de confusão podemos ver que o modelo fez uma boa previsão, como verificado com a accurácia ele acertando acima dos 50% dos dados que temos.

In [40]:
# Calculando valor do Recall
recall = recall_score(y_train, y_pred_train, average='macro')
print("Recall", recall)

Recall 0.5854938533276812


Usando o recall_score nos dados podemos verificar o mesmo valor da accurácia de 58%

# 4) Aplique o modelo aos dados de teste e realize a avaliação dos resultados, da mesma forma que fez acima. Não se esqueça de avaliar com as suas palavras e comparar o desempenho da base treino com a teste.

In [43]:
y_pred_test = naive_bayes.predict(X_test)

In [48]:
accuracy = accuracy_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test) # não usaremos o average='macro' pois não esta balanceado

print("Acurácia:", accuracy)
print("Recall:", recall)

Acurácia: 0.555261363180487
Recall: 0.6479727169382342


Podemos ja verificar alguns valores diferentes Acurácia ja nos mostra um valor um pouco abaixo de 55% e o Recall um valor suporior de 64%

In [46]:
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Nomes das clases
class_name = ['Chun', 'Não Churn']

# Plot da matriz de confusão com Plotly
fig = ff.create_annotated_heatmap(
    z = conf_matrix,
    x = class_name,
    y = class_name,
    colorscale = 'Viridis',
    showscale = True,
)
fig.update_layout(
    title = 'Matriz de Confusão - Teste',
    xaxis_title = 'Predict',
    yaxis_title = 'True',
    font = dict(
        family = 'Arial',
        size = 12,
        color = 'black'
    )
)


na Matriz de confusão ja podemos uma grande diferença, o modelo ja apresenta o acerto só de quem não da churn

# 5) Descreva com suas palavras o projeto desenvolvido nessa atividade e qual o nosso objetivo principal ao aplicarmos o algoritmo de naive bayes a base de crédito.
Utilize pelo menos 4 linhas.

Dica: Caso você ainda esteja tendo dificuldade em visualizar a aplicação dos projetos e objetivo, consulte seus tutores!

Com Naive Bayes podemos verificar que ele apresenta um peso para cada variavel de forma idenpendente assim não mostra um resultado muito bom e eficiente.
Realizando treino na base teste ele performou significamente bem com uma accurácia e recall de 58%, e na matriz de confussão bem balanceada mas no teste ja não mostrou muito bem, com Accurácia de 55% e com um bom reccall de 64% mas isso no mostra que esta acertando somente uma das opcões não um balanceamento eficiente, quando plotamos a matriz de confussão podemos ver claramente um desbalancemento e vemos que esta prevendo somente uma das classes com eficiencia